A lightweight Gen AI prototype using LangChain, FAISS, and OpenAI for semantic search and question-answering over Emma Watson’s UN speech.

In [1]:
!pip install langchain

In [2]:
!pip install openai

In [3]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00


In [4]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [6]:
# we are using pdf doc as input
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.5 MB/s eta 0:00:00


In [7]:
# Save the input file which is transcript of Emma Watson's speech, in the folder
import requests
import io
import pdfplumber  # install via pip install pdfplumber

url = "https://www.un.int/sites/www.un.int/files/IAPR/full-transcript-of-emma-watson.pdf"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to download PDF (status {response.status_code})")

with pdfplumber.open(io.BytesIO(response.content)) as pdf:
    all_text = "\n\n".join(page.extract_text() for page in pdf.pages)

with open("emma_speech_un_transcript.txt", "w", encoding="utf-8") as f:
    f.write(all_text)

In [8]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./emma_speech_un_transcript.txt')
documents = loader.load()

In [9]:
# Text Splitter
#Split the text into chuncks
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

#check the total length of chunks
print(f"Total chunks: {len(docs)}")


Total chunks: 3


In [58]:
# Transformer only required if using hugging face
# Used for Loading the transformer model
# and Converting text into embeddings (numerical vectors)

#!pip install sentence_transformers

In [10]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize embeddings (uses text-embedding-ada-002 by default)
embeddings = OpenAIEmbeddings()

/tmp/ipython-input-10-1550437173.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [11]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.0 MB/s eta 0:00:00


In [12]:
from langchain.vectorstores import FAISS

#using FAISS for db vector stores
db = FAISS.from_documents(docs, embeddings)

# Look at the stored vectors in the raw form
# Number of vectors stored
print(db.index.ntotal)

# Vector for the first document
vec = db.index.reconstruct(0)  # FAISS uses integer IDs
print(vec[:10])  # Show first 10 dimensions of the vector


#vector store data in humar readable form
print(db.index)  # This is the FAISS index itself
print(db.docstore._dict)  # Contains mapping from internal IDs to LangChain Document objects


#no need to embed the query manually
# We used FAISS.from_documents(docs, embeddings), so the returned db(vector store object) already knows what embeddings model we used.
#so, when we use the raw query, langchain automatically embeds it behind the scenes using the same embedding object that we passed before

# Let's try chnging the two queries below and see the difference in the result

# This query below should return the first chunck
query = "what is the name of Emma's campaign?"

# This query below should return the third chunk
#query = "How many girls will be married in the next 16 years"

docs_result = db.similarity_search(query)

3
[-0.02684503 -0.02059751 -0.01423405 -0.00765161  0.01162555 -0.00914586
 -0.02280025  0.00354885 -0.03217797 -0.02416568]
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7ed764111380> >
{'66c51dff-f6d7-43ce-9170-ed0605bbde84': Document(id='66c51dff-f6d7-43ce-9170-ed0605bbde84', metadata={'source': './emma_speech_un_transcript.txt'}, page_content="Full Transcript of Emma Watson's Speech\non Gender Equality at the UN\nEmma Invites All of Us to Fight for Gender Equality\nEmma Watson with UN Secretary General Bank Ki-moon at the launch of the HeForShe\ncampaign in New York City. Eduardo Munoz Alvarez/Stringer\nOn Saturday, September 20, British actor and Goodwill Ambassador for UN Women, Emma\nWatson, gave a smart, important, and moving speech about gender inequality and how to fight\nit. In doing so, she launched the HeForShe initiative, which aims to get men and boys to\npledge to join the feminist fight for gender equality. In the speech M

In [13]:
#print(docs[0].page_content)
print(docs_result[0].page_content)

Full Transcript of Emma Watson's Speech
on Gender Equality at the UN
Emma Invites All of Us to Fight for Gender Equality
Emma Watson with UN Secretary General Bank Ki-moon at the launch of the HeForShe
campaign in New York City. Eduardo Munoz Alvarez/Stringer
On Saturday, September 20, British actor and Goodwill Ambassador for UN Women, Emma
Watson, gave a smart, important, and moving speech about gender inequality and how to fight
it. In doing so, she launched the HeForShe initiative, which aims to get men and boys to
pledge to join the feminist fight for gender equality. In the speech Ms. Watson makes the very
important point that in order for gender equality to be achieved, harmful and destructive
stereotypes of and expectations for masculinity have got to change. Below is the full transcript
of her thirteen-minute speech.
Today we are launching a campaign called for HeForShe. I am reaching out to you because
we need your help. We want to end gender inequality, and to do this, we ne

In [14]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# using FAISS vector store as retriever
retriever = db.as_retriever()

# Creating a QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True  #  to see where the answer came from
)

# Run the query
# Query 1
result = qa_chain({"query": "what did Emma tell herself firmly?"})

# Query 2
#result = qa_chain({"query": "How many girls will be married in the next 16 years"})

print("Answer:", result["result"])

/tmp/ipython-input-14-1303419606.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(),
/tmp/ipython-input-14-1303419606.py:16: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": "what did Emma tell herself firmly?"})


Answer: 
"If not me, who? If not now, when?"


In [17]:
# This query below should return the third chunk
query = "How many girls will be married in the next 16 years"

docs_result = db.similarity_search(query)

print(docs_result[0].page_content)

Both men and women should feel free to be sensitive. Both men and women should feel free to
be strong. It is time that we all perceive gender on a spectrum, instead of two sets of opposing
ideals. If we stop defining each other by what we are not, and start defining ourselves by who
we are, we can all be freer, and this is what HeForShe is about. It’s about freedom.
I want men to take up this mantle so that their daughters, sisters, and mothers can be free
from prejudice, but also so that their sons have permission to be vulnerable and human too,
reclaim those parts of themselves they abandoned, and in doing so, be a more true and
complete version of themselves.
You might be thinking, “Who is this Harry Potter girl, and what is she doing speaking at the
UN?” And, it’s a really good question. I’ve been asking myself the same thing.
All I know is that I care about this problem, and I want to make it better. And, having seen
what I’ve seen, and given the chance, I feel it is my responsibi

In [18]:
# Query 2
result = qa_chain({"query": "How many girls will be married in the next 16 years"})

print("Answer:", result["result"])

Answer:  15.5 million
